## Absolute change in 99th percentile 1-day accumulated precipitation
This notebook generates the text metadata files for the in-land flooding exposure metric `absolute change in 99th percentile 1-day accumulated precipitation`, using data from Cal-Adapt: Analytics Engine data. Because the AE data represents 200+ GB of data, metrics were calculated with a cluster in a high performance computing environment (i.e., a pcluster). 
Please see the processing script `climate_ae_precipitation_accumulation_metrics.py` for full methodological process.

### Step 1: Generate metadata

In [ ]:
import pandas as pd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [ ]:
# options here: climate_extreme_heat_hot_day_metric, climate_extreme_heat_warm_night_metric
df_in = pd.read_csv('climate_flood_exposure_precipitation_metric.csv') # make sure this is in the same folder!
df_in # check

In [ ]:
@append_metadata
def precip_ae_data_process(df, export=False, export_filename=None, varname=''):
    '''
    Reduces the size of the initial daily raw precipitation data in order to streamline compute time.
    Transforms the raw data into the following baseline metrics:
    * Absolute change in 99th percentile 1-day accumulated precipitation
    
    Methods
    -------
    Metric is calculated by pooling data across models and calculating the 99th percentile. 
    See https://github.com/cal-adapt/cae-notebooks/blob/main/exploratory/internal_variability.ipynb
    for reasoning behind data pooling for precipitation model data. 
    
    Parameters
    ----------
    df: pd.DataFrame
        Input data.
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI flooding metric to AWS
        True = will upload resulting df containing CAL CRAI flooding metric to AWS
    export_filename: string
        name of csv file to be uploaded to AWS
    varname: string
        Final metric name, for metadata generation
        
    Script
    ------
    Metric calculation: climate_ae_precipitation_accumulation_metric.py via pcluster run
    Example metric calculation for Alameda county: climate_ae_precipitation_accumulation_metric_example.ipynb
    Metadata generation: climate_ae_precipitation_accumulation_metadata.ipynb
    
    Note
    ----
    Because the climate projections data is on the order of 2.4 TB in size, intermediary
    processed files are not produced for each stage of the metric calculation. All processing
    occurs in a single complete run in the notebook listed above.
    '''
        
    # calculate with 2°C WL
    print('Data transformation: raw projections data retrieved for warming level of 2.0°C, by manually subsetting based on GWL for parent GCM and calculating 30 year average.')
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")

    # historical baseline
    print("Data transformation: historical baseline data retrieved for 1981-2010, averaging across models.")
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")

    # calculate delta signal
    print("Data transformation: snowfall sigal removed from precipitation data to isolate liquid precipitation from total precipitation.")
    print("Data transformation: data clipped to remove 0.1mm to remove trace precipitation.")
    print("Data transformation: leap days removed from historical data to match time periods.")
    print("Data transformation: data pooled across models to increase sample size and drop all singleton dimensions (scenario).")
    print("Data transformation: calculate 99th percentile from pooled data.")
    print("Data transformation: delta signal calculated by taking difference between chronic (2.0°C) and historical baseline.")
    print("Data transformation: non-CA grid points removed from data.")

    # reprojection to census tracts
    print("Data transformation: data transformed from xarray dataset into pandas dataframe.")
    print("Data transformation: data reprojected from Lambert Conformal Conic CRS to CRS 3310.")
        
    # min-max standardization
    print("Data transformation: data min-max standardized with min_max_standardize function.")
    
    # export data as csv
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [export_filename]
        upload_csv_aws(export_filename, bucket_name, directory)
        
        # Check if file exists before attempting to remove it
        if os.path.exists('climate_flood_exposure_precipitation_metric.csv'):
            os.remove('climate_flood_exposure_precipitation_metric') # remove from local to clear up directory
        
        if os.path.exists(export_filename[0]):
            os.remove(export_filename[0])
    
    if export == False:
        print(f'{export_filename} uploaded to AWS.')

In [ ]:
precip_ae_data_process(df_in, export=False, export_filename='climate_flood_exposure_precipitation_metric.csv', varname='test')